# <img style="float: left;" src="https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png">  

### Tracking Pipeline Table Schema


##### Transactions Table
| user_id| time_stamp             |  action_id  | session_id 
|------  | ----------             | ----------- | ---------- 
|   1    | 2016-01-24 10:23:54    |  1          | 1
|   1    | 2016-01-24 10:23:60    |  2          | 1
|   1    | 2016-01-24 10:24:60    |  3          | 1
|   1    | 2016-01-24 10:25:60    |  4          | 1
|   2    | 2016-01-24 10:25:60    |  1          | 2
|   2    | 2016-01-24 10:26:60    |  2          | 2

##### Action Types:

| action_id| action_type         |  
|----------| ----------          |
|   1      | Viewed home page    |
|   2      | Viewed cape page    |
|   3      | Added cape to cart  |
|   4      | Purchased item      |

##### Users
| user_id | first_name| last_nam   | gender | City         | 
|---------| ----------| ---------- | ------ | --------     |
|   1     | Clark     | Kent       |    M   |  Smallville  |
|   2     | Bruce     | Wayne      |    M   |  Gotham      |
|   3     | Barry     | Allen      |    M   |  Central City|
|   4     | Diane     | Prince     |    F   |  Themyscira  |






#### 1) What is the maximum level in the funnel that was reached in each session?

```sql
SELECT session_id, MAX(action_id) AS MAX_DEPTH
FROM transactions
GROUP BY session_id
```

#### 2) What city had the most purchases?

```sql
SELECT u.city, count(*)
FROM transactions t
INNER JOIN users u
ON t.user_id = u.user_id
WHERE action_id = 4
GROUP BY u.city
ORDER BY count(*) DESC
LIMIT 1
```

#### 3) How many users made it to each level of the funnel? (Show the count of each level as a seperate column)

```sql
SELECT
SUM(CASE WHEN action_id = 1 THEN 1 ELSE 0 END) AS ONE,
SUM(CASE WHEN action_id = 2 THEN 1 ELSE 0 END) AS TWO,
SUM(CASE WHEN action_id = 3 THEN 1 ELSE 0 END) AS THREE,
SUM(CASE WHEN action_id = 4 THEN 1 ELSE 0 END) AS FOUR
from transactions
```

#### 4) What percentage of sessions end at each level of the funnel?

### Tracking Users and Churn


##### Transactions Table
| user_id| time_stamp             |  tweet_id   | 
|------  | ----------             | ----------- |  
|   1    | 2016-01-24 10:23:54    |  1          | 
|   2    | 2016-01-24 10:23:60    |  2          | 
|   3    | 2016-01-24 10:24:60    |  3          | 
|   1    | 2016-02-24 10:25:60    |  4          | 
|   2    | 2016-02-24 10:25:60    |  5          | 
|   1    | 2016-03-24 10:26:60    |  6          | 
        
##### Users
|id     | full_name      | login_id   |
|-------| ---------      | -------    |
| 1     | Oscar Grouch   | OG         |
| 2     | Cookie Monster | MyCookie   |
| 3     | Burt Doe       | IHeartErnie|

##### Tweets
|id     | tweet                                  |
|-------|------------------------------          |
| 1     | I need a shower                        |
| 2     | Is it Girl Scout cookie season?        |
| 3     | Eernie, where are you?                 |
| 4     | I can't fit my trash can in the shower.|
| 5     | Does anyone have some milk?            |
| 6     | So fresh and so clean clean            |

##### 3) How many distinct users were there each month? Each month's count should be a seperate column. 

#### 1) What users have posted at least 2 tweets in between Jan 1 and Feb 28?

```sql
SELECT user_id 
FROM posts
WHERE time_stamp between '1/1/2016' and '2/28/2016'
GROUP BY user_id
HAVING count(*) >= 2
```

#### 2) Which user posted the most questions? Include user_id and count. 

```sql
SELECT p.user_id, count(*)
from posts p
JOIN tweets t
ON p.tweet_id = t.id
WHERE t.tweets like ('%?')
GROUP BY p.user_id
ORDER BY 2 DESC
LIMIT 1
```

##### 3) How many distinct users were there each month? Each month's count should be a seperate column. 

```sql
SELECT count(DISTINCT p.user_id) AS JAN_TWEETS
, count(DISTINCT feb.user_id) AS FEB_TWEETS
, count(DISTINCT mar.user_id) AS MAR_TWEETS

FROM posts p
FULL OUTER JOIN 
    (
    SELECT f.user_id 
    FROM posts f
    WHERE f.time_stamp between '2/1/2016' and '2/28/2016'
    ) AS feb
ON p.user_id = feb.user_id
FULL OUTER JOIN
    (
    SELECT m.user_id 
    FROM posts m
    WHERE m.time_stamp between '3/1/2016' and '3/31/2016'
    ) AS mar
ON p.user_id = mar.user_id
WHERE p.time_stamp between '1/1/2016' and '1/31/2016'
```